In [1]:
import numpy as np
import pandas as pd

In [43]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [44]:
def func1():
    dataset = pd.read_csv("groceries2.txt", delimiter='	')
    dataset= dataset.drop_duplicates()

    # I create rows with each couple of products bought together (in the same transaction) 
    df_merged=dataset.set_index('Transaction ID').join(dataset.set_index('Transaction ID'), lsuffix='_1', rsuffix='_2')
    
    # I remove duplicates. 
    # LINE TO BE IMPROVED: THIS LINE TAKES 44% OF THE TIME
    df_merged = df_merged[df_merged['Item_1']>df_merged['Item_2']]
    # END OF LINE TO BE IMPROVED
    
    df_merged = df_merged[['Item_1','Item_2']].groupby(['Item_1','Item_2']).size().reset_index(name='counts')

    # I count the total transactions, and add the 'support' column to the dataset
    total_transactions = dataset['Transaction ID'].nunique()
    df_merged['Support'] = df_merged.counts/total_transactions

    # I modify the groceries table, to count how many transactions for each item
    dataset = dataset.groupby(['Item']).size().reset_index(name='count')
    # I add the support for each item
    dataset['Support'] = dataset['count']/ total_transactions
    # I remove the transaction ID column
    dataset = dataset[['Item','Support']]
    # I set the Item column to be the index (useful for merging later)
    dataset = dataset.set_index('Item')

    # I add the support for item 1 to the dataset
    df_merged= df_merged.merge(dataset.rename({'Support': 'Support Item 1'}, axis=1), left_on='Item_1', right_index=True)
    # I add the support for item 2 to the dataset
    df_merged= df_merged.merge(dataset.rename({'Support': 'Support Item 2'}, axis=1), left_on='Item_2', right_index=True)

    # I add the additional metrics
    df_merged['Confidence Item 1 to Item 2'] = df_merged['Support']/df_merged['Support Item 1']
    df_merged['Confidence Item 2 to Item 1'] = df_merged['Support']/df_merged['Support Item 2']
    df_merged['Lift'] = df_merged['Support']/(df_merged['Support Item 1']*df_merged['Support Item 2'])

    return df_merged

In [45]:
%lprun -f func1 func1()

In [32]:
dataset = pd.read_csv("groceries2.txt", delimiter='	')
dataset= dataset.drop_duplicates()

# I create rows with each couple of products bought together (in the same transaction) 
#df_merged=dataset.set_index('Transaction ID').join(dataset.set_index('Transaction ID'), lsuffix='_1', rsuffix='_2')

In [4]:
df_merged.head()

,Item_1,Item_2
Transaction ID,,
0,WHITE HANGING HEART T-LIGHT HOLDER,WHITE HANGING HEART T-LIGHT HOLDER
0,WHITE HANGING HEART T-LIGHT HOLDER,RED WOOLLY HOTTIE WHITE HEART.
0,WHITE HANGING HEART T-LIGHT HOLDER,KNITTED UNION FLAG HOT WATER BOTTLE
0,WHITE HANGING HEART T-LIGHT HOLDER,SET 7 BABUSHKA NESTING BOXES
0,WHITE HANGING HEART T-LIGHT HOLDER,WHITE METAL LANTERN


In [12]:
df_merged.iloc[0]['Item_1'] > df_merged.iloc[0]['Item_1']

False

In [13]:
from pandas import DataFrame, merge
df1 = DataFrame({'key':[1,1], 'col1':[1,2],'col2':[3,4]})
df2 = DataFrame({'key':[1,1], 'col3':[5,6]})

merge(df1, df2,on='key')[['col1', 'col2', 'col3']]

,col1,col2,col3
0,1,3,5
1,1,3,6
2,2,4,5
3,2,4,6


In [14]:
df1

,key,col1,col2
0,1,1,3
1,1,2,4


In [15]:
df2

,key,col3
0,1,5
1,1,6


In [20]:
dataset.set_index('Transaction ID', inplace=True)

In [24]:
dataset.index

True

In [27]:
from time import time

In [29]:
t0 = time()
merge(dataset, dataset,on='Transaction ID')
t1 = time()

In [30]:
t1 - t0

8.974408864974976

In [33]:
t0 = time()
df_merged=dataset.set_index('Transaction ID').join(dataset.set_index('Transaction ID'), lsuffix='_1', rsuffix='_2')
t1 = time()
t1 - t0

4.396506309509277

In [35]:
C = np.where(df_merged['Item_1'] != df_merged['Item_2'])

In [39]:
df_merged.iloc[C[0]]

,Item_1,Item_2
Transaction ID,,
0,WHITE HANGING HEART T-LIGHT HOLDER,RED WOOLLY HOTTIE WHITE HEART.
0,WHITE HANGING HEART T-LIGHT HOLDER,KNITTED UNION FLAG HOT WATER BOTTLE
0,WHITE HANGING HEART T-LIGHT HOLDER,SET 7 BABUSHKA NESTING BOXES
0,WHITE HANGING HEART T-LIGHT HOLDER,WHITE METAL LANTERN
0,WHITE HANGING HEART T-LIGHT HOLDER,CREAM CUPID HEARTS COAT HANGER
...,...,...
21788,CHILDRENS CUTLERY CIRCUS PARADE,CIRCUS PARADE LUNCH BOX
21788,CHILDRENS CUTLERY CIRCUS PARADE,CHILDRENS CUTLERY SPACEBOY
21788,CHILDRENS CUTLERY CIRCUS PARADE,CHILDREN'S APRON DOLLY GIRL


In [38]:
C[0]

array([       1,        2,        3, ..., 57148384, 57148385, 57148386])

In [2]:
dataset = pd.read_csv("groceries2.txt", delimiter='	')
dataset= dataset.drop_duplicates()

# I create rows with each couple of products bought together (in the same transaction) 
df_merged=dataset.set_index('Transaction ID').join(dataset.set_index('Transaction ID'), lsuffix='_1', rsuffix='_2')

In [41]:
t0 = time()
# I remove duplicates. 
# LINE TO BE IMPROVED: THIS LINE TAKES 44% OF THE TIME
df_merged = df_merged[df_merged['Item_1']>df_merged['Item_2']]
# END OF LINE TO BE IMPROVED

In [3]:
import numba

In [9]:
@numba.jit(nopython=True)
def remove_dupl(item_1, item_2):
    arr = np.empty(len(item_1), dtype=bool)
    for i in range(len(item_1)):
        arr[i] = item_1[i] > item_2[i]
    return arr

In [6]:
from time import time

In [10]:
t0 = time()
indexes = remove_dupl(df_merged['Item_1'].to_numpy(),df_merged['Item_2'].to_numpy())
t1 = time()

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
non-precise type array(pyobject, 1d, C)
During: typing of argument at <ipython-input-9-d42a62cdbcd7> (3)

File "<ipython-input-9-d42a62cdbcd7>", line 3:
def remove_dupl(item_1, item_2):
    arr = np.empty(len(item_1), dtype=bool)
    ^


In [8]:
t1 - t0

58.24459409713745

In [80]:
df_merged.head(10)[x]

,Item_1,Item_2
Transaction ID,,
0,WHITE HANGING HEART T-LIGHT HOLDER,RED WOOLLY HOTTIE WHITE HEART.
0,WHITE HANGING HEART T-LIGHT HOLDER,KNITTED UNION FLAG HOT WATER BOTTLE
0,WHITE HANGING HEART T-LIGHT HOLDER,SET 7 BABUSHKA NESTING BOXES
0,WHITE HANGING HEART T-LIGHT HOLDER,CREAM CUPID HEARTS COAT HANGER
0,WHITE HANGING HEART T-LIGHT HOLDER,GLASS STAR FROSTED T-LIGHT HOLDER
0,RED WOOLLY HOTTIE WHITE HEART.,KNITTED UNION FLAG HOT WATER BOTTLE


In [77]:
df_merged.head(10)['Item_1'].to_numpy()[0] >df_merged.head(10)['Item_2'].to_numpy()[0]

False